Setup date range and weekdays

In [298]:
import random
from datetime import datetime, timedelta, date
import pandas as pd

#Function to create date dataframe and add weekdays
def TT_weekday_picker(start_date, end_date, desired_weekday, closed_dates):
    s = pd.date_range(start_date, end_date, freq='D').to_series()
    d = s.dt.day_name()

    df = pd.DataFrame(data=d)
    df = df.reset_index(level=0)
    df.index = df.index + 1
    df.rename(columns={'index':'date', 0: 'weekday'}, inplace = True)
    #select specific weekday if desired & add dates to exclude
    tt_df = df[df.weekday.isin(desired_weekday) & ~df.date.isin(closed_dates)]

    return tt_df

#Input desired date ranges and days
first_day = '2023-08-01'
last_day = '2023-12-21'
working_days_of_week = ['Tuesday', 'Thursday']
closed_days = ['2023-03-14', '2023-03-15', '2023-06-29', '2023-06-30']

#call function
tt_df = TT_weekday_picker(first_day, last_day, working_days_of_week, closed_days)


Select Tuesdays and Thursdays

In [299]:
#some stats
index = tt_df.index
number_of_rows = len(index)
print(f"The number of kitchen & cleaning days between {start_date} & {end_date} is {number_of_rows}")


The number of kitchen & cleaning days between 2023-08-01 & 2023-12-23 is 42


In [273]:
#rename df for function to come
dates_data = tt_df.drop('weekday', axis=1, inplace=False)

In [281]:
import numpy as np

# Sample data for debugging
people_data = {
    'person': ['Anna R', 'Richard', 'Nikhil', 'Anna L', 'Ebba', 'Pelle', 'Olof', 'Sabina', 'Nathalie', 'Patric', 'Elvira', 'Jörgen', 'Thomas', 'Kajsa', 'Craig', 'Isabel', 'Anna T', 'Nicoline', 'Martin', 'Anastasia', 'Vincent', 'Didem', 'Johanna']
}

#late_people_data = ['Person_14', 'Person_9', 'Person_6'] # Add the names you want to assign as later

people = pd.DataFrame(people_data)
late_people = pd.DataFrame(late_people_data)

#stats
#print(len(people))
#print(len(dates_data))
#print(len(dates_data) // len(people))

In [286]:
#function to evenly distribute each name to as many dates as needed
def assign_people_to_dates(dates_df, people_df):
    num_dates = len(dates_df)
    num_people = len(people_df)
    
    # Calculate the number of people needed for each date (set to 1 in this case)
    people_per_date = 1
    total_people_needed = people_per_date * num_dates
    
    # Repeat the people list to match the total number of assignments
    people_list = people_df['person'].tolist() * (total_people_needed // num_people)
    remaining_people = total_people_needed % num_people
    if remaining_people > 0:
        people_list += people_df['person'].head(remaining_people).tolist()

    # Create DataFrames for morning and evening assignments
    dates_df['person_kitchen'] = np.array_split(people_list[:total_people_needed], num_dates)
    dates_df['person_cleaning'] = np.array_split(people_list[:total_people_needed], num_dates)

    # Concatenate morning and evening assignments
    kitchen_df = dates_df[['date', 'person_kitchen']].explode('person_kitchen')
    cleaning_df = dates_df[['date', 'person_cleaning']].explode('person_cleaning')

      # Shuffle the 'Name' columns
    kitchen_shuffle = np.random.permutation(kitchen_df['person_kitchen'])
    kitchen_df['person_kitchen'] = kitchen_shuffle
    cleaning_shuffle = np.random.permutation(cleaning_df['person_cleaning'])
    cleaning_df['person_cleaning'] = cleaning_shuffle

    # Combine morning and evening DataFrames and drop the intermediate columns
    result_df = kitchen_df.merge(cleaning_df, on = 'date', how = 'inner')

    return result_df

# Call the function with your DataFrames
assigned_df = assign_people_to_dates(dates_data, people)

assigned_df

,date,person_kitchen,person_cleaning
0,2023-08-01,Anna L,Anna T
1,2023-08-03,Anastasia,Anna R
2,2023-08-08,Richard,Anastasia
3,2023-08-10,Martin,Jörgen
4,2023-08-15,Anna T,Nicoline
5,2023-08-17,Johanna,Patric
6,2023-08-22,Vincent,Anna R
7,2023-08-24,Nicoline,Nathalie
8,2023-08-29,Thomas,Kajsa
9,2023-08-31,Nikhil,Anna T


In [289]:
#verify counts
result = assigned_df.groupby('person_kitchen').size().reset_index(name='count')
result2 = assigned_df.groupby('person_cleaning').size().reset_index(name='count')

print(result)
print(result2)

   person_kitchen  count
0       Anastasia      1
1          Anna L      2
2          Anna R      2
3          Anna T      2
4           Craig      2
5           Didem      1
6            Ebba      2
7          Elvira      2
8          Isabel      2
9         Johanna      1
10         Jörgen      2
11          Kajsa      2
12         Martin      2
13       Nathalie      2
14       Nicoline      2
15         Nikhil      2
16           Olof      2
17         Patric      2
18          Pelle      2
19        Richard      2
20         Sabina      2
21         Thomas      2
22        Vincent      1
   person_cleaning  count
0        Anastasia      1
1           Anna L      2
2           Anna R      2
3           Anna T      2
4            Craig      2
5            Didem      1
6             Ebba      2
7           Elvira      2
8           Isabel      2
9          Johanna      1
10          Jörgen      2
11           Kajsa      2
12          Martin      2
13        Nathalie      2
14        

In [290]:
#check how it looks in table

assigned_df.to_csv('lottocheck.csv', index=False)
